<a href="https://colab.research.google.com/github/yousrabougara/WalmartSalesForecasting/blob/main/TabPFN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install sklearn
!pip install scikit-learn==1.5.2

# Install TabPFN
!pip install tabpfn

# TabPFN Community installs optional functionalities around the TabPFN model
# These include post-hoc ensembles, interpretability tools, and more
!git clone https://github.com/PriorLabs/tabpfn-extensions
!pip install -e tabpfn-extensions[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 108.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# Setup Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Markdown, Latex

# Baseline Imports
from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


# TabPFN Imports
from tabpfn import TabPFNClassifier, TabPFNRegressor


import torch

from tabpfn import TabPFNClassifier, TabPFNRegressor


if not torch.cuda.is_available():
    raise SystemError('GPU device not found. For fast training, please enable GPU. See section above for instructions.')

In [ ]:
df=pd.read_csv('Preprocessed_Walmart.csv')

In [ ]:
df.head(5)

,Store_0,Store_1,Store_2,Store_3,Store_4,Store_5,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Season_0,Season_1,Season_2
0,0,0,0,0,0,1,2010-02-05,1.057420,0,-0.995136,-1.713800,1.004175,0.056964,2,2010,0,0,1
1,0,0,0,0,0,1,2010-02-12,1.054348,1,-1.201170,-1.766089,1.007880,0.056964,2,2010,0,0,1
2,0,0,0,0,0,1,2010-02-19,1.001206,0,-1.124178,-1.840166,1.009074,0.056964,2,2010,0,0,1
3,0,0,0,0,0,1,2010-02-26,0.642828,0,-0.760907,-1.737766,1.009849,0.056964,2,2010,0,0,1
4,0,0,0,0,0,1,2010-03-05,0.899914,0,-0.767955,-1.598328,1.010624,0.056964,3,2010,0,1,0


In [ ]:
# Convert 'Date' to datetime format (if it's not already)
df['Date'] = pd.to_datetime(df['Date'])

# Add day of the month and week of the year
df['Day'] = df['Date'].dt.day
df['Week'] = df['Date'].dt.isocalendar().week

In [ ]:
df.head(5)

,Store_0,Store_1,Store_2,Store_3,Store_4,Store_5,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Season_0,Season_1,Season_2,Day,Week
0,0,0,0,0,0,1,2010-02-05,1.057420,0,-0.995136,-1.713800,1.004175,0.056964,2,2010,0,0,1,5,5
1,0,0,0,0,0,1,2010-02-12,1.054348,1,-1.201170,-1.766089,1.007880,0.056964,2,2010,0,0,1,12,6
2,0,0,0,0,0,1,2010-02-19,1.001206,0,-1.124178,-1.840166,1.009074,0.056964,2,2010,0,0,1,19,7
3,0,0,0,0,0,1,2010-02-26,0.642828,0,-0.760907,-1.737766,1.009849,0.056964,2,2010,0,0,1,26,8
4,0,0,0,0,0,1,2010-03-05,0.899914,0,-0.767955,-1.598328,1.010624,0.056964,3,2010,0,1,0,5,9


In [ ]:
features = df.columns.drop(['Weekly_Sales','Date','Year'])
target = 'Weekly_Sales'

X = df[features]
y = df[target]

X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=42 ,  test_size=0.2)

In [ ]:
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNRegressor
from sklearn.metrics import r2_score

# Pas de scaling ni d'encodage des catégories
#num_vars = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Weekly_Sales', 'Lag_1', 'Lag_2', 'Lag_3', 'Lag_4','Day','Week']
#cat_vars = ['Store', 'Season']

# Séparer features et target
features = df.columns.drop(['Weekly_Sales', 'Date'])
target = 'Weekly_Sales'

X = df[features]
y = df[target]

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Entraîner TabPFN
reg = TabPFNRegressor(device='cuda' or 'cpu', random_state=42)  # Choisir 'cuda' si dispo pour accélérer
reg.fit(X_train, y_train)

# Prédictions
y_pred = reg.predict(X_test)

# Évaluation du R²
r2 = r2_score(y_test, y_pred)
print(f"TabPFN R² Score: {r2:.4f}")


TabPFN R² Score: 0.9838
